In [11]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# Set up the LM
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
dspy.settings.configure(lm=turbo)


class CreateHaiku(dspy.Signature):
    """Create a one sentence haiku based on the topic."""

    topic = dspy.InputField()
    context = dspy.InputField()
    answer = dspy.OutputField(desc="a sentence with a haiku")


class HaikuPipeline(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.generate_haiku = dspy.Predict(CreateHaiku)

    def forward(self, topic):
        context = ""
        prediction = self.generate_haiku(topic=topic, context=context)
        return dspy.Prediction(topic=topic, answer=prediction.answer, context=context)


from dspy.teleprompt import BootstrapFewShot


# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    # answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_PM


# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

examples = [
    dspy.Example(
        topic="trees",
        answer="Golden leaves falling / Nature's dance with the wind / Serene and peaceful.",
    ),
    dspy.Example(
        topic="fun", answer="Joyful laughter / Fun bubbles up / Childhood memories."
    ),
    dspy.Example(
        topic="hope",
        answer="Hope blooms / Like a flower in spring / Brighten up our days.",
    ),
    dspy.Example(
        topic="money",
        answer="Money piles high / A symbol of success / Negotiations begin.",
    ),
    # dspy.Example(topic="age", answer="Aging gracefully / Reflection of life / Nourishes growth."),
    # dspy.Example(topic="stupidity", answer="Stupidity never ends / Knowledge is power / Ignorance is not an option."),
    # dspy.Example(topic="poop", answer="Poop comes out of the pot / In the bathroom bush / Needs to go potty."),
    # dspy.Example(topic="nerds", answer="Nerds rule the world / Intelligence is power / Geek pride forever."),
    # dspy.Example(topic="bananas", answer="Bananas are delicious / Eating them makes me happy / Oh), the joy of fruit."),
    # dspy.Example(topic="whistles", answer="Whistling in the dark / Sounds so good in my ear / Peaceful and serene."),
    # dspy.Example(topic="beauty", answer="Beauty is in the eye of the beholder / A creation of nature / Timeless and evergreen."),
]

trainset = [x.with_inputs("topic") for x in examples]
compiled_haiku = teleprompter.compile(HaikuPipeline(), trainset=trainset)

# my_topic = "castles"

# pred = compiled_haiku(my_topic)

# print(f"Question: {my_topic}")
# print(f"Predicted Answer: {pred.answer}")

# turbo.inspect_history(n=1)

100%|██████████| 4/4 [00:00<00:00, 4449.01it/s]
